In [1]:
#cm_sb_cdummies
import json
import logging
import views

logging.basicConfig(
    level=logging.DEBUG,
    #level=logging.INFO, # uncomment this and comment debug above for less yelling in red
    format=views.config.LOGFMT,
)
# DATASETS is a dictionary of Dataset objects.
from views import DATASETS
# These are the building blocks of the modelling interface
from views import Ensemble, Model, Downsampling, Period
# These are model specifications from the specfiles
from views.specs.models import cm as model_specs_cm, pgm as model_specs_pgm
from views.specs.periods import get_periods, get_periods_by_name
# Utils
from views.utils import db, io, data as datautils
from views.utils.data import assign_into_df
from views.apps.pipeline.models_cm import all_cm_models_by_name
from views.apps.pipeline.models_pgm import all_pgm_models_by_name
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
dataset = views.DATASETS["cm_africa_imp_0"]
df = dataset.df
run_id = "d_2020_04_01"

/home/kyle/miniconda3/envs/views2/lib/python3.8/site-packages/libpysal/examples/remotes.py:26: UserWarning: Remote data sets not available. Check connection.
  warnings.warn("Remote data sets not available. Check connection.")
[2020-11-19 17:32:25,247] - views.utils.io:107 - DEBUG - Loading YAML from /home/kyle/code/Views2/OpenViEWS2/views/specs/periods/periods.yaml
[2020-11-19 17:32:25,275] - views.utils.io:107 - DEBUG - Loading YAML from /home/kyle/code/Views2/OpenViEWS2/views/specs/periods/periods.yaml
[2020-11-19 17:32:25,297] - views.utils.io:65 - DEBUG - Reading parquet at /home/kyle/code/Views2/OpenViEWS2/storage/data/datasets/cm_africa_imp_0.parquet with cols None
[2020-11-19 17:32:25,768] - views.utils.io:72 - DEBUG - Finished reading parquet from /home/kyle/code/Views2/OpenViEWS2/storage/data/datasets/cm_africa_imp_0.parquet.


In [2]:
periods = get_periods(run_id) # as a list
periods_by_name = get_periods_by_name(run_id) # as a dict
period_a = periods_by_name["A"]
period_b = periods_by_name["B"]
period_c = periods_by_name["C"]
period_c

[2020-11-19 17:32:28,924] - views.utils.io:107 - DEBUG - Loading YAML from /home/kyle/code/Views2/OpenViEWS2/views/specs/periods/periods.yaml
[2020-11-19 17:32:28,945] - views.utils.io:107 - DEBUG - Loading YAML from /home/kyle/code/Views2/OpenViEWS2/views/specs/periods/periods.yaml


Period(name='C', train_start=121, train_end=480, predict_start=483, predict_end=520)

In [3]:
model_from_pipeline_spec = all_cm_models_by_name["cm_sb_reign_coups"]
models=[model_from_pipeline_spec]

In [4]:
for model in models:
    model.periods = periods

In [6]:
model_from_pipeline_spec

{
  "name": "cm_sb_reign_coups",
  "col_outcome": "greq_25_ged_best_sb",
  "cols_features": [
    "reign_couprisk",
    "reign_pctile_risk"
  ],
  "steps": [
    1,
    3,
    6,
    9,
    12,
    18,
    24,
    30,
    36,
    38
  ],
  "periods": [
    {
      "name": "A",
      "train_start": 121,
      "train_end": 396,
      "predict_start": 397,
      "predict_end": 432
    },
    {
      "name": "B",
      "train_start": 121,
      "train_end": 432,
      "predict_start": 433,
      "predict_end": 468
    },
    {
      "name": "C",
      "train_start": 121,
      "train_end": 480,
      "predict_start": 483,
      "predict_end": 520
    }
  ],
  "outcome_type": "prob",
  "estimators": {
    "name": "cm_sb_reign_coups",
    "initial_estimator": {
      "base_estimator": {
        "criterion": "gini",
        "splitter": "best",
        "max_depth": null,
        "min_samples_split": 2,
        "min_samples_leaf": 1,
        "min_weight_fraction_leaf": 0.0,
        "max_feature

In [5]:
for model in models:
    model.fit_estimators(df, populate_extras = False)

[2020-11-19 17:33:40,468] - views.apps.model.api:441 - INFO - Fitting estimators for cm_sb_reign_coups
[2020-11-19 17:33:40,469] - views.apps.model.api:444 - DEBUG - Fitting cm_sb_reign_coups for period A step 1
[2020-11-19 17:33:40,480] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sb_reign_coups
[2020-11-19 17:33:40,481] - views.apps.model.api:422 - DEBUG - cm_sb_reign_coups downsampled away 0
[2020-11-19 17:33:40,482] - views.apps.model.api:424 - DEBUG - Fitting cm_sb_reign_coups on 14850 rows
[2020-11-19 17:33:40,482] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sb_reign_coups
[2020-11-19 17:34:19,344] - views.apps.model.api:116 - DEBUG - Saving cm_sb_reign_coups A 1 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_reign_coups_A_1.joblib
[2020-11-19 17:34:39,040] - views.apps.model.api:119 - DEBUG - cm_sb_reign_coups saved to /home/kyle/code/Views2/OpenViEWS2/storage/mo

[2020-11-19 17:42:56,518] - views.apps.model.api:119 - DEBUG - cm_sb_reign_coups saved to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_reign_coups_A_36.joblib
[2020-11-19 17:42:56,530] - views.apps.model.api:444 - DEBUG - Fitting cm_sb_reign_coups for period A step 38
[2020-11-19 17:42:56,535] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sb_reign_coups
[2020-11-19 17:42:56,535] - views.apps.model.api:422 - DEBUG - cm_sb_reign_coups downsampled away 0
[2020-11-19 17:42:56,536] - views.apps.model.api:424 - DEBUG - Fitting cm_sb_reign_coups on 12852 rows
[2020-11-19 17:42:56,536] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sb_reign_coups
[2020-11-19 17:43:38,666] - views.apps.model.api:116 - DEBUG - Saving cm_sb_reign_coups A 38 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_reign_coups_A_38.joblib
[2020-11-19 17:43:59,257] - views.apps.model.api:119 - DEBUG - cm

[2020-11-19 17:53:38,062] - views.apps.model.api:116 - DEBUG - Saving cm_sb_reign_coups B 30 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_reign_coups_B_30.joblib
[2020-11-19 17:54:01,486] - views.apps.model.api:119 - DEBUG - cm_sb_reign_coups saved to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_reign_coups_B_30.joblib
[2020-11-19 17:54:01,503] - views.apps.model.api:444 - DEBUG - Fitting cm_sb_reign_coups for period B step 36
[2020-11-19 17:54:01,509] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sb_reign_coups
[2020-11-19 17:54:01,510] - views.apps.model.api:422 - DEBUG - cm_sb_reign_coups downsampled away 0
[2020-11-19 17:54:01,510] - views.apps.model.api:424 - DEBUG - Fitting cm_sb_reign_coups on 14904 rows
[2020-11-19 17:54:01,511] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sb_reign_coups
[2020-11-19 17:54:46,504] - views.apps.model.api:116 - DEBUG - Sa

[2020-11-19 18:05:39,518] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sb_reign_coups
[2020-11-19 18:06:41,744] - views.apps.model.api:116 - DEBUG - Saving cm_sb_reign_coups C 24 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_reign_coups_C_24.joblib
[2020-11-19 18:07:11,355] - views.apps.model.api:119 - DEBUG - cm_sb_reign_coups saved to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_reign_coups_C_24.joblib
[2020-11-19 18:07:11,382] - views.apps.model.api:444 - DEBUG - Fitting cm_sb_reign_coups for period C step 30
[2020-11-19 18:07:11,389] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sb_reign_coups
[2020-11-19 18:07:11,390] - views.apps.model.api:422 - DEBUG - cm_sb_reign_coups downsampled away 0
[2020-11-19 18:07:11,390] - views.apps.model.api:424 - DEBUG - Fitting cm_sb_reign_coups on 17820 rows
[2020-11-19 18:07:11,391] - views.apps.model.api:137 - DEBUG - Ge

In [ ]:
for model in models:
    df_pred = model.predict(df)
    # assign_into_df takes care to only overwrite rows with actual values
    # This way we can keep all periods in the same df
    # It's also idempotent, no joining, so run as many times as you like. 
    df = assign_into_df(df_to=df, df_from=df_pred)
    
    df_pred = model.predict_calibrated(
        df=df.fillna(0), 
        period_calib=period_a,
        period_test=period_b,)
        
    df = assign_into_df(df_to=df, df_from=df_pred)
    
    df_pred = model.predict_calibrated(
        df=df.fillna(0), 
        period_calib=period_b,
        period_test=period_c,
    )
    df = assign_into_df(df_to=df, df_from=df_pred)
   

In [ ]:
for model in models:
    model.evaluate(df)

In [ ]:
for model in models:
    print(model.name)
    #print(model.scores)
    print("EVAL SCORES:")
    print(json.dumps(model.scores, indent=2))
    print("FEATURE_IMPORTANCES")
    print(json.dumps(model.extras.feature_importances, indent=2))
    print("#"*80)